In [1]:
import argparse, time, os
import random

from tqdm import tqdm
import copy, math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from pyproj import Proj, Transformer, Geod
import xarray
import xrspatial.slope
from bokeh.models import PrintfTickFormatter, FixedTicker
import datashader as ds
import holoviews as hv
from holoviews import opts
from holoviews.operation.datashader import datashade, rasterize, inspect_points
import hvplot.xarray
import hvplot.pandas
import rioxarray

hv.extension('bokeh')

In [2]:
from bokeh.themes.theme import Theme
# Custom Bokeh theme for improved colorbar title styling
theme = Theme(json={
    'attrs': {
        'Title': {'align': 'center', 'text_font_size': '15px'},  # Larger and centered title
        'Axis': {'axis_label_text_font_style': 'normal'},         # No italic labels
        'Legend': {'title_text_font_style': 'normal'},
        'ColorBar': {'title_text_font_size': '16pt',              # Larger colorbar title
                     'title_text_font_style': 'normal',
                    'major_label_text_font_size': '13pt'}
    }
})

hv.renderer('bokeh').theme = theme

In [3]:
bedmap_folder = f"/media/maffe/nvme/polar_ice_thickness_data/bedmap"

# Find all .csv files in the directory
files_bedmap = [
    os.path.join(root, file)
    for root, _, files in os.walk(bedmap_folder)
    for file in files if file.endswith(".csv")
]

# Process each file
total_files = len(files_bedmap)
print(f"We are dealing with {total_files} files from BedMap")

We are dealing with 151 files from BedMap


In [4]:
# Columns to check for. Also let's rename the column names to values on the right
required_columns_bedmap = {
    'longitude (degree_east)': 'LON',
    'latitude (degree_north)': 'LAT',
    'date': 'date',
    'surface_altitude (m)': 'surface_altitude (m)',
    'land_ice_thickness (m)': 'THICK',
    'bedrock_altitude (m)': 'bedrock_altitude (m)'
}

In [5]:
# List to store dataframes
bedmap_list_csv = []
    
for i, file in tqdm(enumerate(files_bedmap), total=total_files, leave=True):

    try:    
        # Read the CSV file, skipping the header
        df_file_csv = pd.read_csv(file, skiprows=18, low_memory=False, dtype={'date': 'string'})

        # Check if required columns exist
        missing_columns = [col for col in required_columns_bedmap.keys() if col not in df_file_csv.columns]
        if missing_columns:
            raise ValueError(f"Missing columns: {missing_columns}")

        # Select only the required columns
        #df_file_csv = df_file_csv[required_columns_bedmap]
        df_file_csv = df_file_csv[required_columns_bedmap.keys()].rename(columns=required_columns_bedmap)
        
        # Append to the list of dataframes
        bedmap_list_csv.append(df_file_csv)

    except Exception as e:
        print(f"Error processing file '{file}': {e}")

100%|█████████████████████████████████████████| 151/151 [01:15<00:00,  2.00it/s]


In [6]:
# Concatenate all dataframes into one
combined_bedmap_df = pd.concat(bedmap_list_csv, axis=0, ignore_index=True)
print(f"Original BedMap no. points: {len(combined_bedmap_df)}.")

Original BedMap no. points: 95892117.


In [7]:
# Data cleaning

# NOTE: this could be further refined. In particular, some crossing tracks sometimes appear not consistent

# 1) remove rows where THICK is -9999
combined_bedmap_df = combined_bedmap_df[~combined_bedmap_df[['THICK']].isin([-9999]).any(axis=1)
]
# 2) Remove too small ice thickness data
combined_bedmap_df = combined_bedmap_df[combined_bedmap_df['THICK'] > 0.5]
print(f"no. points: {len(combined_bedmap_df)}.")

no. points: 81087281.


In [8]:
print(combined_bedmap_df.head(5))

         LON        LAT   date  surface_altitude (m)   THICK  \
0  76.889142 -69.876749  -9999               -9999.0  1046.6   
1  76.893603 -69.876762  -9999               -9999.0  1058.5   
2  76.899026 -69.876753  -9999               -9999.0  1061.1   
3  76.904100 -69.876424  -9999               -9999.0  1063.7   
4  76.909194 -69.876374  -9999               -9999.0  1069.8   

   bedrock_altitude (m)  
0               -9999.0  
1               -9999.0  
2               -9999.0  
3               -9999.0  
4               -9999.0  


In [9]:
bedmap = combined_bedmap_df[['LON', 'LAT', 'THICK']]
bedmap_gdf = gpd.GeoDataFrame(
    bedmap,
    geometry=gpd.points_from_xy(bedmap['LON'], bedmap['LAT']), crs="EPSG:4326").to_crs("EPSG:3031")
bedmap_gdf['EAST'] = bedmap_gdf.geometry.x.values
bedmap_gdf['NORTH'] = bedmap_gdf.geometry.y.values
print(bedmap_gdf.head(5))
print(bedmap_gdf.crs)

         LON        LAT   THICK                        geometry          EAST  \
0  76.889142 -69.876749  1046.6  POINT (2150724.686 500918.979)  2.150725e+06   
1  76.893603 -69.876762  1058.5  POINT (2150762.268 500751.208)  2.150762e+06   
2  76.899026 -69.876753  1061.1  POINT (2150810.656 500547.878)  2.150811e+06   
3  76.904100 -69.876424  1063.7  POINT (2150890.832 500365.723)  2.150891e+06   
4  76.909194 -69.876374  1069.8  POINT (2150940.759 500175.761)  2.150941e+06   

           NORTH  
0  500918.979443  
1  500751.208165  
2  500547.878047  
3  500365.723321  
4  500175.761230  
EPSG:3031


In [10]:
points_bedmap = hv.Points(bedmap_gdf, ['EAST', 'NORTH'], 'THICK') # HoloViews Points object

formatter = PrintfTickFormatter(format='%i')

# Rasterize the points
rasterized_bedmap = rasterize(points_bedmap,aggregator=ds.mean('THICK')).opts(
    cmap=plt.colormaps['turbo'],
    cnorm='eq_hist',
    width=700,
    height=700,
    colorbar=True,
    xaxis=None,            # Remove x-axis
    yaxis=None,            # Remove y-axis
    colorbar_opts={
        'title': 'Ice Thickness measurements [m]',
        'formatter': formatter,  # Apply custom formatter to the colorbar,
        "ticker": FixedTicker(ticks=[500, 1000, 1500, 2000, 2500, 3000, 4500])
    },
    colorbar_position='bottom',
)


# Display the plot
hv.output(rasterized_bedmap)

:DynamicMap   []
   :Image   [EAST,NORTH]   (EAST_NORTH THICK)

In [11]:
FOLDER_AN_VELOCITY = f"/media/maffe/nvme/Antarctica_NSIDC/velocity/NSIDC-0754/"
FOLDER_AN_THICKNESS = f"/media/maffe/nvme/Antarctica_NSIDC/thickness/NSIDC-0756/"
FOLDER_AN_RACMO = f"/media/maffe/nvme/racmo/antarctica_racmo2.3p2/2km/"

file_an_v = f"{FOLDER_AN_VELOCITY}antarctic_ice_vel_phase_map_v01.nc"
file_an_smb = f"{FOLDER_AN_RACMO}smb_antarctica_mean_1979_2021_RACMO23p2_gf.nc"
file_an_bedmac = f"{FOLDER_AN_THICKNESS}BedMachineAntarctica-v3.nc"

In [12]:
# Open Antarctica arrays
an_v = rioxarray.open_rasterio(file_an_v)
an_smb = rioxarray.open_rasterio(file_an_smb)
an_bedmac = rioxarray.open_rasterio(file_an_bedmac)

print(an_v['VX'].rio.resolution())
print(an_v['VY'].rio.resolution())
print(an_bedmac['thickness'].rio.resolution())
print(an_bedmac['surface'].rio.resolution())
print(an_smb.rio.resolution())

an_bedmac['thickness'] = an_bedmac['thickness'].where(an_bedmac['thickness'] != 0.0)

# Calculate the slope
res_elevation = an_bedmac['surface'].rio.resolution()[0] # 500 m
#an_bedmac['slope'] = xrspatial.slope(an_bedmac['surface'].squeeze())
dz_dy, dz_dx = np.gradient(an_bedmac['surface'].squeeze(), res_elevation, res_elevation)
slope_raw = np.sqrt(dz_dx**2 + dz_dy**2)
an_bedmac['slope'] = (('y', 'x'), slope_raw)

nodata_v = an_v['VX'].rio.nodata

an_vx = an_v['VX']
an_vy = an_v['VY']

an_vx.values = np.where(an_vx.values == nodata_v, np.nan, an_vx.values)
an_vy.values = np.where(an_vy.values == nodata_v, np.nan, an_vy.values)

an_v['V'] = (an_vx**2 + an_vy**2)**0.5

assert an_vx.rio.crs == an_vy.rio.crs == an_smb.rio.crs == an_bedmac.rio.crs, 'Different crs.'

(450.0, -450.0)
(450.0, -450.0)
(500.0, -500.0)
(500.0, -500.0)
(2000.0, -2000.0)


In [13]:
# Interpolate arrays linearly
eastings_ar = xarray.DataArray(bedmap_gdf['EAST'])
northings_ar = xarray.DataArray(bedmap_gdf['NORTH'])

elev_interp_data = an_bedmac['surface'].interp(y=northings_ar, x=eastings_ar, method='linear').data.squeeze()
ith_interp_data = an_bedmac['thickness'].interp(y=northings_ar, x=eastings_ar, method='linear').data.squeeze()
slope_interp_data = an_bedmac['slope'].interp(y=northings_ar, x=eastings_ar, method='linear').data.squeeze()
v_interp_data = an_v['V'].interp(y=northings_ar, x=eastings_ar, method='linear').data.squeeze()
smb_interp_data = an_smb.interp(y=northings_ar, x=eastings_ar, method='linear').data.squeeze()

In [14]:
# Plug in dataset

bedmap_gdf['v'] = v_interp_data
bedmap_gdf['ith_bm'] = ith_interp_data
bedmap_gdf['smb'] = smb_interp_data
bedmap_gdf['z'] = elev_interp_data
bedmap_gdf['s'] = slope_interp_data

In [15]:
print(f"Before dropping nans: {len(bedmap_gdf)} points")
bedmap_gdf = bedmap_gdf.dropna()
print(f"After dropping nans: {len(bedmap_gdf)} points")

Before dropping nans: 81087281 points
After dropping nans: 79890423 points


In [16]:
# Check no invalid points
invalid_count = (~bedmap_gdf.geometry.is_valid).sum()
assert invalid_count==0, f"Invalid geometries: {invalid_count}"

In [17]:
cols = ['THICK', 'smb', 'z', 's', 'v']
corr_matrix = bedmap_gdf[cols].corr()
print(corr_matrix)

          THICK       smb         z         s         v
THICK  1.000000 -0.364856  0.742594 -0.346988 -0.240941
smb   -0.364856  1.000000 -0.454212  0.368349  0.228904
z      0.742594 -0.454212  1.000000 -0.207117 -0.359341
s     -0.346988  0.368349 -0.207117  1.000000  0.006415
v     -0.240941  0.228904 -0.359341  0.006415  1.000000


In [22]:
# Save
save = False
if save:
    bedmap_gdf.to_parquet(f"{bedmap_folder}/bedmap_train.parquet", engine='pyarrow', index=False)
    print(f"Combined BedMap dataframe saved as parquet: {len(bedmap_gdf)} points.")

In [18]:
# Plot

x = an_bedmac['slope'].hvplot.image(
    x='x', y='y',
    rasterize=True,
    dynamic=True,
    #logz=True,
    clim=(0, .3),
).opts(
    cmap='binary',
    width=700,
    height=700,
    xaxis=None,
    yaxis=None,
    title="",
    colorbar_opts={
        'title': 'Surface slope [unitless]',
    },
    colorbar_position='bottom',
    tools=[]
)

# HoloViews Points object
points = hv.Points(bedmap_gdf, ['EAST', 'NORTH'], 's')

# Rasterize the points
#rasterized_points = rasterize(points,aggregator=ds.mean('s')).opts(
#    cmap=plt.colormaps['turbo'],
#    #logz=True,
#    clim=(0, 2.5),
#    colorbar=False,
#    tools=['hover'],
#)


x #* rasterized_points

:DynamicMap   []
   :Image   [x,y]   (slope)